In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cxData_shielding = pd.read_excel("./cxData.xlsx", sheet_name = "shielding materials")
cxData_ss304 = pd.read_excel("./cxData.xlsx", sheet_name = "ASS304")
cxData_conc = pd.read_excel("./cxData.xlsx", sheet_name = "Concrete(2)")

# Cx Data 곱하기

<!-- Compound 수밀도 $\times$ 존재비율 (wt.%) $\times$ Cx -->

### H2O  
-> H 값 몇 배 ? O 값 몇 배?  
-> H의 수밀도 $\times$ $\sigma$ $\times$ 2 + O의 수밀도 $\times$ $\sigma$ = H2O의 거시적 단면적

In [3]:
# Be-9  : el.scattering capture 
# H-1   : el.scattering.1 capture.1
# O-16  : el.scattering.2 capture.2
# C-12  : el.scattering.3 capture.3

# print(cxData_shielding) 
# print(cxData_ss304)
# print(cxData_conc)

# H2O

### Load Cx Data

In [4]:
Na = 6.023 * 10**(23)
barn = 10**(-24)

H1_Energylst = cxData_shielding["H-1"].drop(0)
H1_elasticScattering = cxData_shielding["el.scattering.1"].drop(0)
H1_capture = cxData_shielding["capture.1"].drop(0)
H1_numberdensity = (Na / 1.00794) * 0.000084
print(H1_numberdensity) # #/cm3

O16_Energylst = cxData_shielding["O-16"].drop(0)
O16_elasticScattering = cxData_shielding["el.scattering.2"].drop(0)
O16_capture = cxData_shielding["capture.2"].drop(0)
O16_numberdensity = (Na / 15.999) * 0.001332
print(O16_numberdensity)

5.019465444371687e+19
5.014460903806487e+19


In [20]:
cxData_shielding

,Be-9,el.scattering,capture,H-1,el.scattering.1,capture.1,O-16,el.scattering.2,capture.2,C-12,el.scattering.3,capture.3
0,Energy(eV),XS(b),XS(b),Energy(eV),XS(b),XS(b),Energy(eV),XS(b),XS(b),Energy(eV),XS(b),XS(b)
1,0.00001,118.253,0.504006,0.00001,1173.11,16.7299,0.00001,54.7724,0.008545,0.00001,79.1113,0.194257
2,0.00001,116.451,0.496311,0.00001,1155.2,16.4744,0.00001,53.9395,0.008415,0.00001,77.9072,0.191291
3,0.000011,114.73,0.488958,0.000011,1138.09,16.2303,0.000011,53.1438,0.00829,0.000011,76.7566,0.188457
4,0.000011,113.083,0.481922,0.000011,1121.72,15.9968,0.000011,52.3824,0.008171,0.000011,75.6557,0.185745
...,...,...,...,...,...,...,...,...,...,...,...,...
2981,NaN,NaN,NaN,NaN,NaN,NaN,142000000,0.170414,0,NaN,NaN,NaN
2982,NaN,NaN,NaN,NaN,NaN,NaN,144000000,0.164747,0,NaN,NaN,NaN
2983,NaN,NaN,NaN,NaN,NaN,NaN,146000000,0.15908,0,NaN,NaN,NaN
2984,NaN,NaN,NaN,NaN,NaN,NaN,148000000,0.153479,0,NaN,NaN,NaN


### Energy에 따라서 새로운 H2O Cx 생성

- 거시적 단면적 다 더할거라매요 
- $\Sigma$_H2O = 2 * $\Sigma$_H1 + 1 * $\Sigma$_O-16

In [5]:
H1_macroElastic = H1_elasticScattering * H1_numberdensity * barn
H1_macroCapture = H1_capture * H1_numberdensity * barn
H1_macroCx = H1_macroElastic + H1_macroCapture

In [6]:
O16_macroElastic = O16_elasticScattering * O16_numberdensity * barn
O16_macroCapture = O16_capture * O16_numberdensity * barn
O16_macroCx = O16_macroElastic + O16_macroCapture

### Data 준비 완료

- H1의 에너지에 따라서, H2O Cx를 만들고 싶음

### O-16의 Elastic Capture 데이터 전처리 해줘야 함.
### 우리는 H1의 에너지 bin에 맞춰서 사용할거임.

In [21]:
def calcuO16Value(energy: float, whichCx: str) -> float:
    """
    energy를 받아서, 그 에너지에 대한 O16의 cx 데이터를 가져오는 거임
    """
    prevIdx, prevEnergy = 1., 0.

    if whichCx == "Elastic":
        tmpMacroCxData = O16_macroElastic

    elif whichCx == "Capture":
        tmpMacroCxData = O16_macroCapture

    for idx, tmpEnergy in O16_Energylst.items():
        if energy <= tmpEnergy: ### tmp Energy 사용 
            ### Interpolation 
            ### 사용 가능한 값 -> idx, tmpEnergy
            ### 이전 값: idx-1, prevEnergy 
            ### 현재 값: idx, tmpEnergy
            prevCxValue = tmpMacroCxData[prevIdx]
            curCxValue = tmpMacroCxData[idx]
            KE = energy
            curEnergy = tmpEnergy
            
            ### 오류가 난 부분이 20 MeV 넘는 초과하는 energy가 들어오면, 이 부분이 안 돈다는 건데
            resultCxValue = \
                prevCxValue + (KE-prevEnergy) * ((curCxValue-prevCxValue)/(curEnergy-prevEnergy))

            # print(KE, tmpEnergy, resultCxValue)
            break;
            

        else:
            prevIdx = idx
            prevEnergy = tmpEnergy
        
    return resultCxValue

In [26]:
### initializing
H2O_macroElastic = []
H2O_macroCapture = []

### Macro Elastic Cx
for idx, energy in H1_Energylst.items():
    """
    energy => eV 단위로 들어와 있고요 idx 는 1 부터 시작을 해요
    """
    
    if (np.isnan(energy)):
        break;

    # macroElastic = H1_macroElastic[idx] * 2 + O16_macroElastic[idx]
    tmpmacroElastic = H1_macroElastic[idx] * 2 + calcuO16Value(energy, "Elastic")
    tmpmacroCapture = H1_macroCapture[idx] * 2 + calcuO16Value(energy, "Capture")
    H2O_macroElastic.append(tmpmacroElastic)
    H2O_macroCapture.append(tmpmacroCapture)

H2O_macroElastic = np.array(H2O_macroElastic)
H2O_macroCapture = np.array(H2O_macroCapture)

In [24]:
# for elastic, capture in zip(H2O_macroElastic, H2O_macroCapture):
#     print(elastic, capture)

In [31]:
H2O_microElastic = []
H2O_microCapture = []

H2O_numberdensity = (Na / 18.015248) * 0.000998207
print(H2O_numberdensity)

H2O_microElastic = H2O_macroElastic / H2O_numberdensity
H2O_microCapture = H2O_macroCapture / H2O_numberdensity

3.3372844831222972e+19


In [36]:
# for elastic, capture in zip(H2O_microElastic, H2O_microCapture):
#     print(elastic / barn, capture / barn)

In [58]:
# # print(len(H1_Energylst.dropna()))
energylst = H1_Energylst.dropna()
# total = H2O_microCapture + H2O_microElastic
# # print(len(total))
# plt.scatter(x = energylst, y = H2O_microCapture + H2O_microElastic)
# plt.yscale("log")
# plt.xscale("log")
# plt.xlim(10000,10000000)
# plt.ylim(1.e-23, 5.e-22)

# O2

In [53]:
# O16_Energylst = cxData_shielding["O-16"].drop(0)
# O16_elasticScattering = cxData_shielding["el.scattering.2"].drop(0)
# O16_capture = cxData_shielding["capture.2"].drop(0)
# O16_numberdensity = (Na / 15.999) * 0.001332

O2_microElastic = O16_elasticScattering * 2
O2_microCapture = O16_capture * 2

In [55]:
# for i, j in zip(O2_microElastic, O2_microCapture):
#     print(i, j)

### Export Data
- 1E-05       0.503984     Lin-Lin
- 1.10304E-05 0.479868     Lin-Lin
- 1.21669E-05 0.456906     Lin-Lin

In [90]:
df1 = pd.DataFrame(H2O_microElastic)
df2 = pd.DataFrame(H2O_microCapture)
df3 = pd.DataFrame(O2_microElastic)
df4 = pd.DataFrame(O2_microCapture)

f = open("./H2O_elastic.txt", "w")
for idx, cxVal in df1[0].items():
    result = str(energylst[idx+1]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()

f = open("./H2O_capture.txt", "w")
for idx, cxVal in df2[0].items():
    result = str(energylst[idx+1]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()

f = open("./O2_elastic.txt", "w")
for idx, cxVal in df3["el.scattering.2"].items():
    result = str(O16_Energylst[idx]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()

f = open("./O2_capture.txt", "w")
for idx, cxVal in df4["capture.2"].items():
    result = str(O16_Energylst[idx]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()

In [260]:
### Graphite

C12_Energylst = cxData_shielding["C-12"].drop(0).dropna()
C12_elasticScattering = cxData_shielding["el.scattering.3"].drop(0).dropna()
C12_capture = cxData_shielding["capture.3"].drop(0).dropna()

C12_microElastic = C12_elasticScattering
C12_microCapture = C12_capture

df1 = pd.DataFrame(C12_microElastic)
df2 = pd.DataFrame(C12_microCapture)

f = open("./Graphite_elastic.txt", "w")
for idx, cxVal in df1["el.scattering.3"].items():
    result = str(C12_Energylst[idx]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()

f = open("./Graphite_capture.txt", "w")
for idx, cxVal in df2["capture.3"].items():
    result = str(C12_Energylst[idx]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()

# Outer Material
- ss304
- concrete

In [236]:
cxData_ss304 = pd.read_excel("./cxData.xlsx", sheet_name = "ASS304")

C 6000 6000 0.000400 0.001830 0.000160  
Si 14000 14000 0.005000 0.009781 0.000858  
P 15031 15000 0.000230 0.000408 0.000036  
S 16000 16000 0.000150 0.000257 0.000023  
Cr 24000 24000 0.190000 0.200762 0.017605  
Mn 25055 25000 0.010000 0.010001 0.000877  
Fe 26000 26000 0.701730 0.690375 0.060538  
Ni 28000 28000 0.092500 0.086587 0.007593  

In [144]:
# print(cxData_ss304.columns)
C12_wf  = 0.000400
Si28_wf = 0.005000
P31_wf  = 0.000230
S32_wf  = 0.000150
Cr52_wf = 0.190000
Mn55_wf = 0.010000
Fe56_wf = 0.701720
Ni58_wf = 0.092500
# compositionFraction = [Cr52_wf, Mn55_wf, Si28_wf, C12_wf, P31_wf, S32_wf, Ni58_wf, Fe56_wf]
ss304_compositionFraction = [C12_wf, Si28_wf, P31_wf, S32_wf, Cr52_wf, Mn55_wf, Fe56_wf, Ni58_wf]
sum(ss304_compositionFraction)

1.0

In [207]:
# cxData_ss304 = pd.read_excel("./cxData.xlsx", sheet_name = "ASS304")

ss304_energylst = []

# ss304_energylst.append(cxData_ss304["C-12"].drop(0).dropna()); ss304_energylst.append(cxData_ss304["C-12"].drop(0).dropna())
# ss304_energylst.append(cxData_ss304["Si-28"].drop(0).dropna()); ss304_energylst.append(cxData_ss304["Si-28"].drop(0).dropna())
# ss304_energylst.append(cxData_ss304["P-31"].drop(0).dropna()); ss304_energylst.append(cxData_ss304["P-31"].drop(0).dropna())
# ss304_energylst.append(cxData_ss304["S-32"].drop(0).dropna()); ss304_energylst.append(cxData_ss304["S-32"].drop(0).dropna())
# ss304_energylst.append(cxData_ss304["Cr-52"].drop(0).dropna()); ss304_energylst.append(cxData_ss304["Cr-52"].drop(0).dropna())
# ss304_energylst.append(cxData_ss304["Mn-55"].drop(0).dropna()); ss304_energylst.append(cxData_ss304["Mn-55"].drop(0).dropna())
# ss304_energylst.append(cxData_ss304["Fe-56"].drop(0).dropna()); ss304_energylst.append(cxData_ss304["Fe-56"].drop(0).dropna())
# ss304_energylst.append(cxData_ss304["Ni-58"].drop(0).dropna()); ss304_energylst.append(cxData_ss304["Ni-58"].drop(0).dropna())

ss304_standardEnergyLst = cxData_ss304["P-31"].drop(0).dropna()
ss304_energylst.append(cxData_ss304["C-12"].drop(0).dropna()) 
ss304_energylst.append(cxData_ss304["Si-28"].drop(0).dropna())
ss304_energylst.append(cxData_ss304["P-31"].drop(0).dropna()) 
ss304_energylst.append(cxData_ss304["S-32"].drop(0).dropna()) 
ss304_energylst.append(cxData_ss304["Cr-52"].drop(0).dropna()) 
ss304_energylst.append(cxData_ss304["Mn-55"].drop(0).dropna()) 
ss304_energylst.append(cxData_ss304["Fe-56"].drop(0).dropna()) 
ss304_energylst.append(cxData_ss304["Ni-58"].drop(0).dropna()) 

# ss304_energylst # 1 - 32091
# ss304_energylst

In [225]:
def calcuSS304CxValue(energy: float, dataEnergyLst, data) -> float:
    """
    energy를 받아서, 그 에너지에 대한 O16의 cx 데이터를 가져오는 거임
    """
    prevIdx, prevEnergy = 1., 0.

    for idx, tmpEnergy in dataEnergyLst.items():
        if energy <= tmpEnergy: ### tmp Energy 사용 
            ### Interpolation 
            ### 사용 가능한 값 -> idx, tmpEnergy
            ### 이전 값: idx-1, prevEnergy 
            ### 현재 값: idx, tmpEnergy
            prevCxValue = data[prevIdx]
            curCxValue = data[idx]
            KE = energy
            curEnergy = tmpEnergy
            
            # print("found at: ", KE, tmpEnergy)
            ### 오류가 난 부분이 20 MeV 넘는 초과하는 energy가 들어오면, 이 부분이 안 돈다는 건데

            resultCxValue = \
                prevCxValue + (KE-prevEnergy) * ((curCxValue-prevCxValue)/(curEnergy-prevEnergy))

            # if energy == 14000000:
            #     print("---Calculated value is :", idx, resultCxValue)
            # print("--- value is: ", resultCxValue)
            
            # print(KE, tmpEnergy, resultCxValue)
            
            break;

        else:
            prevIdx = idx
            prevEnergy = tmpEnergy
        
    return resultCxValue

In [226]:
ss304_compositionCxData = [] ### 2차원 배열임 (elastic, capture)
### C12 0
ss304_compositionCxData.append(
    (cxData_ss304["el.scattering.3"].drop(0).dropna(),
    cxData_ss304["capture.3"].drop(0).dropna())
 )

### Si28 1
ss304_compositionCxData.append(
    (cxData_ss304["el.scattering.2"].drop(0).dropna(),
    cxData_ss304["capture.2"].drop(0).dropna())
 )

### P31 2
ss304_compositionCxData.append(
    (cxData_ss304["el.scattering.4"].drop(0).dropna(),
    cxData_ss304["capture.4"].drop(0).dropna())
 )

### S32 3
ss304_compositionCxData.append(
    (cxData_ss304["el.scattering.5"].drop(0).dropna(),
    cxData_ss304["capture.5"].drop(0).dropna())
 )

### Cr-52 4
ss304_compositionCxData.append(
    (cxData_ss304["el.scattering"].drop(0).dropna(),
    cxData_ss304["capture"].drop(0).dropna())
 )

### Mn-55
ss304_compositionCxData.append(
    (cxData_ss304["el.scattering.1"].drop(0).dropna(),
    cxData_ss304["capture.1"].drop(0).dropna())
 )

### Fe-56
ss304_compositionCxData.append(
    (cxData_ss304["el.scattering.8"].drop(0).dropna(),
    cxData_ss304["capture.8"].drop(0).dropna())
 )

### Mi-58
ss304_compositionCxData.append(
    (cxData_ss304["el.scattering.6"].drop(0).dropna(),
    cxData_ss304["capture.6"].drop(0).dropna())
 )


In [227]:
ss304_microElastic = []
ss304_microCapture = []

for energy in ss304_standardEnergyLst:
    if energy >= 20_000_000:
        break;
    tmpElasticSum = 0.
    tmpCaptureSum = 0.
    for idx, wt in enumerate(ss304_compositionFraction):
        # idx = 0, wt = C-12
        tmpElastic = calcuSS304CxValue(energy = energy, data = ss304_compositionCxData[idx][0], dataEnergyLst = ss304_energylst[idx]) * wt # elastic
        tmpCapture = calcuSS304CxValue(energy = energy, data = ss304_compositionCxData[idx][1], dataEnergyLst = ss304_energylst[idx]) * wt # capture

        tmpElasticSum += tmpElastic
        tmpCaptureSum += tmpCapture
        
        # if energy == 14000000:
        #     print("result: ", energy, wt,tmpElastic)

    ss304_microElastic.append(tmpElasticSum)
    ss304_microCapture.append(tmpCaptureSum)

# C12_wf  = 0.000400
# Si28_wf = 0.005000
# P31_wf  = 0.000230
# S32_wf  = 0.000150
# Cr52_wf = 0.190000
# Mn55_wf = 0.010000
# Fe56_wf = 0.701720
# Ni58_wf = 0.092500

In [228]:
# for (energy, cxVal) in list(zip(ss304_standardEnergyLst, ss304_microElastic)):
#     print(energy, cxVal)

In [229]:
# # 14000000 1.1577814411 ss304 검증
# for wt in ss304_compositionFraction:
#     print(wt)
# # C12_wf  = 0.000400
# # Si28_wf = 0.005000
# # P31_wf  = 0.000230
# # S32_wf  = 0.000150
# # Cr52_wf = 0.190000
# # Mn55_wf = 0.010000
# # Fe56_wf = 0.701720
# # Ni58_wf = 0.092500

In [230]:
# e1 = 0.819214
# e2 = 0.66193
# e3 = 0.9625
# e4 = 0.76331
# e5 = 1.06275
# e6 = 1.1542
# e7 = 1.18095
# e8 = 1.207
# elasticValidate = np.array([e1, e2, e3, e4, e5, e6, e7, e8])

In [231]:
# a1 = 0.000116
# a2 = 0.00044
# a3 = 0.0003
# a4 = 0.00000324
# a5 = 0.000803
# a6 = 0.000618
# a7 = 0.000791
# a8 = 0.000857
# absorptionValidate = np.array([a1, a2, a3, a4, a5, a6, a7, a8])

In [232]:
# ss304_compositionFraction = np.array(ss304_compositionFraction)

# result = elasticValidate * ss304_compositionFraction
# result.sum()

### ss 304 is done

In [234]:
df1 = pd.DataFrame(ss304_microElastic)
df2 = pd.DataFrame(ss304_microCapture)

f = open("./ss304_elastic.txt", "w")
for idx, cxVal in df1[0].items():
    result = str(ss304_standardEnergyLst[idx+1]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()

f = open("./ss304_capture.txt", "w")
for idx, cxVal in df2[0].items():
    result = str(ss304_standardEnergyLst[idx+1]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()

# Concrete

In [245]:
cxData_conc = pd.read_excel("./cxData.xlsx", sheet_name = "Concrete(2)")

In [246]:
# print(cxData_ss304.columns)
# print(cxData_conc.columns)
H1_wf       = 0.004530 # none
O16_wf      = 0.512600 # 1
Na23_wf     = 0.015270 # 7
Al27_wf     = 0.035550 # ss304_10
Si28_wf     = 0.360360 # 2
Ca40_wf     = 0.057910 # 6
Fe56_wf     = 0.013780 # 4
conc_compositionFraction = [H1_wf, O16_wf, Na23_wf, Al27_wf, Si28_wf, Ca40_wf, Fe56_wf]
sum(conc_compositionFraction)

1.0

In [247]:
conc_energylst = []

conc_standardEnergyLst = cxData_conc["Na-23"].drop(0).dropna()
conc_energylst.append(cxData_conc["H-1"].drop(0).dropna()) 
conc_energylst.append(cxData_conc["O-16"].drop(0).dropna())
conc_energylst.append(cxData_conc["Na-23"].drop(0).dropna()) 
conc_energylst.append(cxData_ss304["Al-27"].drop(0).dropna()) 
conc_energylst.append(cxData_conc["Si-28"].drop(0).dropna()) 
conc_energylst.append(cxData_conc["Ca-40"].drop(0).dropna()) 
conc_energylst.append(cxData_conc["Fe-56"].drop(0).dropna()) 

In [248]:
def calcuCONCCxValue(energy: float, dataEnergyLst, data) -> float:
    """
    energy를 받아서, 그 에너지에 대한 O16의 cx 데이터를 가져오는 거임
    """
    prevIdx, prevEnergy = 1., 0.

    for idx, tmpEnergy in dataEnergyLst.items():
        if energy <= tmpEnergy: ### tmp Energy 사용 
            ### Interpolation 
            ### 사용 가능한 값 -> idx, tmpEnergy
            ### 이전 값: idx-1, prevEnergy 
            ### 현재 값: idx, tmpEnergy
            prevCxValue = data[prevIdx]
            curCxValue = data[idx]
            KE = energy
            curEnergy = tmpEnergy
            
            # print("found at: ", KE, tmpEnergy)
            ### 오류가 난 부분이 20 MeV 넘는 초과하는 energy가 들어오면, 이 부분이 안 돈다는 건데

            resultCxValue = \
                prevCxValue + (KE-prevEnergy) * ((curCxValue-prevCxValue)/(curEnergy-prevEnergy))

            # if energy == 14000000:
            #     print("---Calculated value is :", idx, resultCxValue)
            # print("--- value is: ", resultCxValue)
            
            # print(KE, tmpEnergy, resultCxValue)
            
            break;

        else:
            prevIdx = idx
            prevEnergy = tmpEnergy
        
    return resultCxValue

In [249]:
conc_compositionCxData = [] ### 2차원 배열임 (elastic, capture)

# H1_wf       = 0.004530 # none
# O16_wf      = 0.512600 # 1
# Na23_wf     = 0.015270 # 7
# Al27_wf     = 0.035550 # ss304_10
# Si28_wf     = 0.360360 # 2
# Ca40_wf     = 0.057910 # 6
# Fe56_wf     = 0.013780 # 4

### H1 0
conc_compositionCxData.append(
    (cxData_conc["el.scattering"].drop(0).dropna(),
    cxData_conc["capture"].drop(0).dropna())
 )

### O16 1
conc_compositionCxData.append(
    (cxData_conc["el.scattering.1"].drop(0).dropna(),
    cxData_conc["capture.1"].drop(0).dropna())
 )

### Na23 7
conc_compositionCxData.append(
    (cxData_conc["el.scattering.7"].drop(0).dropna(),
    cxData_conc["capture.7"].drop(0).dropna())
 )

### Al27 ss304_10
conc_compositionCxData.append(
    (cxData_ss304["el.scattering.10"].drop(0).dropna(),
    cxData_ss304["capture.10"].drop(0).dropna())
 )

### Si28 2
conc_compositionCxData.append(
    (cxData_conc["el.scattering.2"].drop(0).dropna(),
    cxData_conc["capture.2"].drop(0).dropna())
 )

### Ca40 6
conc_compositionCxData.append(
    (cxData_conc["el.scattering.6"].drop(0).dropna(),
    cxData_conc["capture.6"].drop(0).dropna())
 )

### Fe-56
conc_compositionCxData.append(
    (cxData_conc["el.scattering.4"].drop(0).dropna(),
    cxData_conc["capture.4"].drop(0).dropna())
 )


In [250]:
conc_microElastic = []
conc_microCapture = []

for energy in conc_standardEnergyLst:
    if energy >= 20_000_000:
        break;
    tmpElasticSum = 0.
    tmpCaptureSum = 0.
    for idx, wt in enumerate(conc_compositionFraction):
        # idx = 0, wt = C-12
        tmpElastic = calcuCONCCxValue(energy = energy, data = conc_compositionCxData[idx][0], dataEnergyLst = conc_energylst[idx]) * wt # elastic
        tmpCapture = calcuCONCCxValue(energy = energy, data = conc_compositionCxData[idx][1], dataEnergyLst = conc_energylst[idx]) * wt # capture

        tmpElasticSum += tmpElastic
        tmpCaptureSum += tmpCapture
        
        # if energy == 14000000:
        #     print("result: ", energy, wt,tmpElastic)

    conc_microElastic.append(tmpElasticSum)
    conc_microCapture.append(tmpCaptureSum)

In [252]:
# for (energy, cxVal) in list(zip(conc_standardEnergyLst, conc_microElastic)):
#     print(energy, cxVal)

In [ ]:
# 14000000 0.8401939257899999

In [254]:
df1 = pd.DataFrame(conc_microElastic)
df2 = pd.DataFrame(conc_microCapture)

f = open("./conc_elastic.txt", "w")
for idx, cxVal in df1[0].items():
    result = str(conc_standardEnergyLst[idx+1]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()

f = open("./conc_capture.txt", "w")
for idx, cxVal in df2[0].items():
    result = str(conc_standardEnergyLst[idx+1]) + "\t" + str(cxVal) + "\t" + "Lin-Lin\n"
    # print(result)
    f.write(result)
f.close()